In [162]:
# EDA
import pandas as pd
import pingouin as pg
import plotly.express as px
import plotly.figure_factory as ff  
import matplotlib.pyplot as plt

# Machine Learning
from sklearn.model_selection import cross_validate, StratifiedKFold, cross_val_predict, cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay

# Otimizacao de hiperparametros
import optuna

In [163]:
# Carregar Dataset
df_segmento = pd.read_csv('./datasets/segmento_clientes.csv')

### EDA

In [164]:
# Visualizar os dados
df_segmento.head(10)

,atividade_economica,faturamento_mensal,numero_de_funcionarios,localizacao,idade,inovacao,segmento_de_cliente
0,Comércio,713109.95,12,Rio de Janeiro,6,1,Bronze
1,Comércio,790714.38,9,São Paulo,15,0,Bronze
2,Comércio,1197239.33,17,São Paulo,4,9,Silver
3,Indústria,449185.78,15,São Paulo,6,0,Starter
4,Agronegócio,1006373.16,15,São Paulo,15,8,Silver
5,Serviços,1629562.41,16,Rio de Janeiro,11,4,Silver
6,Serviços,771179.95,13,Vitória,0,1,Starter
7,Serviços,707837.61,16,São Paulo,10,6,Silver
8,Comércio,888983.66,17,Belo Horizonte,10,1,Bronze
9,Indústria,1098512.64,13,Rio de Janeiro,9,3,Bronze


In [165]:
# # Estrutuda do dataset
df_segmento.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   atividade_economica     500 non-null    object 
 1   faturamento_mensal      500 non-null    float64
 2   numero_de_funcionarios  500 non-null    int64  
 3   localizacao             500 non-null    object 
 4   idade                   500 non-null    int64  
 5   inovacao                500 non-null    int64  
 6   segmento_de_cliente     500 non-null    object 
dtypes: float64(1), int64(3), object(3)
memory usage: 27.5+ KB


In [166]:
# Valores possiveis - variaveis categoricas
df_segmento['atividade_economica'].unique()

array(['Comércio', 'Indústria', 'Agronegócio', 'Serviços'], dtype=object)

In [167]:
# Valores possiveis - variaveis categoricas
df_segmento['localizacao'].unique()

array(['Rio de Janeiro', 'São Paulo', 'Vitória', 'Belo Horizonte'],
      dtype=object)

In [168]:
# Valores possiveis - variaveis categoricas
df_segmento['segmento_de_cliente'].unique()

array(['Bronze', 'Silver', 'Starter', 'Gold'], dtype=object)

In [169]:
# Valores possiveis - variaveis categoricas
df_segmento['inovacao'].unique()

array([1, 0, 9, 8, 4, 6, 3, 7, 5, 2])

In [170]:
# Distribuicaoda variavel segmento de cliente (Target)
contagem_target = df_segmento.value_counts('segmento_de_cliente')
contagem_target

segmento_de_cliente
Silver     260
Bronze     202
Starter     22
Gold        16
Name: count, dtype: int64

In [171]:
# Criar uma lista ordenada do target
list_segmentos = ['Starter', 'Bronze', 'Silver', 'Gold']

In [172]:
# Distribuicao da variavel target - contagem
px.bar(contagem_target, color=contagem_target.index, category_orders={'segmento_de_cliente': list_segmentos})

In [173]:
# Distribuicao da variavel target - percentual
percentual_target = contagem_target / len(df_segmento) * 100
px.bar(percentual_target, color=percentual_target.index, category_orders={'segmento_de_cliente': list_segmentos})

In [174]:
# Distribuicao da variavel localizacao - percentual
percentual_localizacao = df_segmento['localizacao'].value_counts() / len(df_segmento) * 100
px.bar(percentual_localizacao, color=percentual_localizacao.index)

In [175]:
# Distribuicao da variavel atividade economica - percentual
percentual_atividade = df_segmento['atividade_economica'].value_counts() / len(df_segmento) * 100
px.bar(percentual_atividade, color=percentual_atividade.index)

In [176]:
# Distribuicao da variavel inovacao - percentual
percentual_inovacao = df_segmento['inovacao'].value_counts() / len(df_segmento) * 100
px.bar(percentual_inovacao, color=percentual_inovacao.index)

In [177]:
# Tabela de contigencia entre localizacao e target (segmento de cliente)
crosstab_localizacao = pd.crosstab(df_segmento['localizacao'], df_segmento['segmento_de_cliente'], margins=True)[list_segmentos].reset_index()

tabela_localizacao = ff.create_table(crosstab_localizacao)

# Mostrar a tabela crostab
tabela_localizacao.show()

In [178]:
# Tabela de contigencia entre atividade_economica e target (segmento de cliente)
crosstab_atividade = pd.crosstab(df_segmento['atividade_economica'], df_segmento['segmento_de_cliente'], margins=True)[list_segmentos].reset_index()
tabela_atividade = ff.create_table(crosstab_atividade)

# Mostrar a tabela crostab
tabela_atividade.show()

In [179]:
# Tabela de contigencia entre inovacao e target (segmento de cliente)
crosstab_inovacao = pd.crosstab(df_segmento['inovacao'], df_segmento['segmento_de_cliente'], margins=True)[list_segmentos].reset_index()

tabela_inovacao = ff.create_table(crosstab_inovacao)

# Mostrar a tabela crostab
tabela_inovacao.show()

In [180]:
# Distribuicao idade da empresa
px.histogram(df_segmento, x='idade')

In [181]:
# Distribuicao faturamento mensal
px.histogram(df_segmento, x='faturamento_mensal')

In [182]:
# BoxPlot - idade da empresa por segmento de cliente
px.box(df_segmento, x='segmento_de_cliente', y='idade', color='segmento_de_cliente', category_orders={'segmento_de_cliente': list_segmentos})

In [183]:
# BoxPlot - faturamento da empresa por segmento de cliente
px.box(df_segmento, x='segmento_de_cliente', y='faturamento_mensal', color='segmento_de_cliente', category_orders={'segmento_de_cliente': list_segmentos})

## Correlacao entre segmento_de_cliente e localizacao

In [184]:
# Teste de Qui-quadrado de Pearson
# H0 - as variaveis sao independentes
# H1 - as variaveis nao sao independentes
# Se p-value > 0.05, aceita a hipotese nula, caso contrario rejeita
valor_esperado, valor_observado, estatisticas = pg.chi2_independence(df_segmento, 'segmento_de_cliente', 'localizacao')


c:\Users\marci\.virtualenvs\ml-decision-tree-classification-lPBLbI5A\Lib\site-packages\pingouin\contingency.py:151: UserWarning:

Low count on observed frequencies.

c:\Users\marci\.virtualenvs\ml-decision-tree-classification-lPBLbI5A\Lib\site-packages\pingouin\contingency.py:151: UserWarning:

Low count on expected frequencies.



In [185]:
# Valor esperado
# E a frequencia que seria esperado se nao houve associacao entre as variaveis
# E calculado utilizando a distribuicao assumida no teste qui-quadrado
valor_esperado

localizacao,Belo Horizonte,Rio de Janeiro,São Paulo,Vitória
segmento_de_cliente,,,,
Bronze,44.844,52.924,48.884,55.348
Gold,3.552,4.192,3.872,4.384
Silver,57.720,68.120,62.920,71.240
Starter,4.884,5.764,5.324,6.028


In [186]:
# Valor observado
# E a frequencia real dos dados coletados
valor_observado

localizacao,Belo Horizonte,Rio de Janeiro,São Paulo,Vitória
segmento_de_cliente,,,,
Bronze,39,62,45,56
Gold,4,3,5,4
Silver,63,60,65,72
Starter,5,6,6,5


In [187]:
# Estatisticas
estatisticas.round(5)

,test,lambda,chi2,dof,pval,cramer,power
0,pearson,1.00000,5.19335,9.0,0.81714,0.05884,0.11369
1,cressie-read,0.66667,5.19198,9.0,0.81726,0.05883,0.11367
2,log-likelihood,0.00000,5.19713,9.0,0.81680,0.05886,0.11374
3,freeman-tukey,-0.50000,5.20798,9.0,0.81581,0.05892,0.11390
4,mod-log-likelihood,-1.00000,5.22494,9.0,0.81428,0.05902,0.11414
5,neyman,-2.00000,5.27777,9.0,0.80945,0.05932,0.11490


As variaveis localizacao e segmento_de_cliente sao independentes.

## Correlacao entre segmento_de_cliente e atividade_economica

In [188]:
# Teste de Qui-quadrado de Pearson
# H0 - as variaveis sao independentes
# H1 - as variaveis nao sao independentes
# Se p-value > 0.05, aceita a hipotese nula, caso contrario rejeita
valor_esperado, valor_observado, estatisticas = pg.chi2_independence(df_segmento, 'segmento_de_cliente', 'atividade_economica')


c:\Users\marci\.virtualenvs\ml-decision-tree-classification-lPBLbI5A\Lib\site-packages\pingouin\contingency.py:151: UserWarning:

Low count on observed frequencies.

c:\Users\marci\.virtualenvs\ml-decision-tree-classification-lPBLbI5A\Lib\site-packages\pingouin\contingency.py:151: UserWarning:

Low count on expected frequencies.

c:\Users\marci\.virtualenvs\ml-decision-tree-classification-lPBLbI5A\Lib\site-packages\scipy\stats\_stats_py.py:7409: RuntimeWarning:

divide by zero encountered in power

c:\Users\marci\.virtualenvs\ml-decision-tree-classification-lPBLbI5A\Lib\site-packages\scipy\stats\_stats_py.py:7409: RuntimeWarning:

invalid value encountered in multiply

c:\Users\marci\.virtualenvs\ml-decision-tree-classification-lPBLbI5A\Lib\site-packages\scipy\stats\_stats_py.py:7406: RuntimeWarning:

divide by zero encountered in divide



In [189]:
# Valor esperado
# E a frequencia que seria esperado se nao houve associacao entre as variaveis
# E calculado utilizando a distribuicao assumida no teste qui-quadrado
valor_esperado

atividade_economica,Agronegócio,Comércio,Indústria,Serviços
segmento_de_cliente,,,,
Bronze,47.672,56.156,49.288,48.884
Gold,3.776,4.448,3.904,3.872
Silver,61.360,72.280,63.440,62.920
Starter,5.192,6.116,5.368,5.324


In [190]:
# Valor observado
# E a frequencia real dos dados coletados
valor_observado

atividade_economica,Agronegócio,Comércio,Indústria,Serviços
segmento_de_cliente,,,,
Bronze,47,56,50,49
Gold,7,5,4,0
Silver,57,75,63,65
Starter,7,3,5,7


In [191]:
# Estatisticas
estatisticas.round(5)

,test,lambda,chi2,dof,pval,cramer,power
0,pearson,1.00000,9.96998,9.0,0.35292,0.08153,0.18948
1,cressie-read,0.66667,10.60995,9.0,0.30339,0.08410,0.20060
2,log-likelihood,0.00000,13.54637,9.0,0.13940,0.09503,0.25375
3,freeman-tukey,-0.50000,NaN,9.0,NaN,NaN,NaN
4,mod-log-likelihood,-1.00000,inf,9.0,0.00000,inf,NaN
5,neyman,-2.00000,NaN,9.0,NaN,NaN,NaN


As variaveis atividade_economica e segmento de cliente sao independentes.

## Correlacao entre inovacao e atividade_economica

In [192]:
# Teste de Qui-quadrado de Pearson
# H0 - as variaveis sao independentes
# H1 - as variaveis nao sao independentes
# Se p-value > 0.05, aceita a hipotese nula, caso contrario rejeita
valor_esperado, valor_observado, estatisticas = pg.chi2_independence(df_segmento, 'segmento_de_cliente', 'inovacao')


c:\Users\marci\.virtualenvs\ml-decision-tree-classification-lPBLbI5A\Lib\site-packages\pingouin\contingency.py:151: UserWarning:

Low count on observed frequencies.

c:\Users\marci\.virtualenvs\ml-decision-tree-classification-lPBLbI5A\Lib\site-packages\pingouin\contingency.py:151: UserWarning:

Low count on expected frequencies.

c:\Users\marci\.virtualenvs\ml-decision-tree-classification-lPBLbI5A\Lib\site-packages\scipy\stats\_stats_py.py:7409: RuntimeWarning:

divide by zero encountered in power

c:\Users\marci\.virtualenvs\ml-decision-tree-classification-lPBLbI5A\Lib\site-packages\scipy\stats\_stats_py.py:7409: RuntimeWarning:

invalid value encountered in multiply

c:\Users\marci\.virtualenvs\ml-decision-tree-classification-lPBLbI5A\Lib\site-packages\scipy\stats\_stats_py.py:7406: RuntimeWarning:

divide by zero encountered in divide



In [193]:
# Valor esperado
# E a frequencia que seria esperado se nao houve associacao entre as variaveis
# E calculado utilizando a distribuicao assumida no teste qui-quadrado
valor_esperado

inovacao,0,1,2,3,4,5,6,7,8,9
segmento_de_cliente,,,,,,,,,,
Bronze,21.008,23.028,22.624,19.392,17.372,17.372,21.816,21.412,18.988,18.988
Gold,1.664,1.824,1.792,1.536,1.376,1.376,1.728,1.696,1.504,1.504
Silver,27.040,29.640,29.120,24.960,22.360,22.360,28.080,27.560,24.440,24.440
Starter,2.288,2.508,2.464,2.112,1.892,1.892,2.376,2.332,2.068,2.068


In [194]:
# Valor observado
# E a frequencia real dos dados coletados
valor_observado

inovacao,0,1,2,3,4,5,6,7,8,9
segmento_de_cliente,,,,,,,,,,
Bronze,36,44,32,22,12,14,15,12,9,6
Gold,0,0,0,0,0,3,0,5,4,4
Silver,10,5,20,25,30,25,38,36,34,37
Starter,6,8,4,1,1,1,1,0,0,0


In [195]:
# Estatisticas
estatisticas.round(5)

,test,lambda,chi2,dof,pval,cramer,power
0,pearson,1.00000,164.29399,27.0,0.0,0.33095,0.99850
1,cressie-read,0.66667,165.49946,27.0,0.0,0.33216,0.99861
2,log-likelihood,0.00000,181.48878,27.0,0.0,0.34784,0.99951
3,freeman-tukey,-0.50000,NaN,27.0,NaN,NaN,NaN
4,mod-log-likelihood,-1.00000,inf,27.0,0.0,inf,NaN
5,neyman,-2.00000,NaN,27.0,NaN,NaN,NaN


As variaveis inovacao e segmento de cliente nao sao independentes.